In [9]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pycep_correios
from geopy.geocoders import Nominatim
from geopy.geocoders import TomTom
from geopy.extra.rate_limiter import RateLimiter

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# conexão com o banco de dados SQL Server
engine = create_engine(
  os.environ.get('DATABASE_URL_SQL_SERVER'),
  connect_args = {"TrustServerCertificate": "yes"},
  echo=False
)

# conexão postgres
engine_postgres = create_engine(
  os.environ.get('DATABASE_URL_SQL_POSTGRES'),
  echo=False
)

# conexão mariadb/mysql
engine_mysql = create_engine(
  os.environ.get('DATABASE_URL_SQL_MYSQL'),
  echo=False
)

file_path = os.path.join('postos_saude_brasil.csv')
df = pd.read_csv(file_path)
df.head()

,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL


In [22]:
# cria o campo endereço completo para facilitar a chamada à API
df['ENDERECO_COMPLETO'] = df['LOGRADOURO'] + ', ' + df['BAIRRO'] + ', ' + df['MUNICIPIO'] + ', ' + df['UF'] + ', ' + 'BRASIL'
# inclui novas colunas
df['latitude'] = None
df['longitude'] = None
df['address'] = None
df['point'] = None

df.head()

,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO,ENDERECO_COMPLETO,latitude,longitude,address,point
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO,"RUA 07 QUEDAS, CENTRO, ACRELANDIA, AC, BRASIL",None,None,None,None
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL,"BR 364 KM 114, ZONA RURAL, ACRELANDIA, AC, BRASIL",None,None,None,None
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO,"AVENIDA PARANA, CENTRO, ACRELANDIA, AC, BRASIL",None,None,None,None
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL,"BR 364 KM 90 RAMAL GRANADA KM 16, ZONA RURAL, ...",None,None,None,None
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL,"AC 475 VILA REDENCAOI, ZONA RURAL, ACRELANDIA,...",None,None,None,None


In [38]:
geolocator_nominatim = Nominatim(user_agent="info_vacinas")
geolocator_tomtom = TomTom(api_key='wvuAFmBut64DLjQqey19XtfuMXZilzbj', user_agent="info_vacinas")
geocode_nominatim = RateLimiter(geolocator_nominatim.geocode, min_delay_seconds=0)
geocode_tomtom = RateLimiter(geolocator_tomtom.geocode, min_delay_seconds=0)
tamanho = len(df)

print(f'Processando {str(tamanho).zfill(5)} registros')
for index, row in df.iterrows():
  # tenta fazer a localização usando o Nominatim
  location = geolocator_nominatim.geocode(row['ENDERECO_COMPLETO'])
  # se a localização não retorna nada, vai para o próximo
  if not location:
    location = geolocator_tomtom.geocode(row['ENDERECO_COMPLETO'])
    if not location:
      continue
  
  # exibe apenas de 500 em 500 para saber como está o processo
  if index%500 == 0:
    print(f'Processando {str(index+1).zfill(5)} de {str(tamanho).zfill(5)} registros')

  row['latitude'] = location.latitude
  row['longitude'] = location.longitude
  row['address'] = location.address
  row['point'] = location.point

df.head()  

Processando 46821 registros
Processando 00001 de 46821 registros
{'type': 'Street', 'id': 'BR/STR/p0/976599', 'score': 9.1880216599, 'matchConfidence': {'score': 0.6483194250725391}, 'address': {'streetName': 'Rua Sete Quedas', 'municipality': 'Acrelândia', 'countrySubdivision': 'Acre', 'countryCode': 'BR', 'country': 'Brasil', 'countryCodeISO3': 'BRA', 'freeformAddress': 'Rua Sete Quedas, Acrelândia', 'localName': 'Acrelândia'}, 'position': {'lat': -10.07561, 'lon': -67.05003}, 'viewport': {'topLeftPoint': {'lat': -10.0666, 'lon': -67.05951}, 'btmRightPoint': {'lat': -10.08458, 'lon': -67.04124}}}
Processando 00001 de 46821 registros
UF                                                                  AC
MUNICIPIO                                                   ACRELANDIA
ESTABELECIMENTO      UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA
LOGRADOURO                                               RUA 07 QUEDAS
NUMERO                                                             284
BA